Práctica 4: Entrenamiento de redes neuronales
===
Autores: Lidia Concepción Echeverría y Francisco Ponce Belmonte
---

In [1]:
from scipy.io import loadmat
import numpy as np
import math
import matplotlib.pyplot as plt
import scipy.optimize as opt

Carga de datos, utilizando el mismo dataset de la práctica anterior.

Inicializamos ya los valores dependientes de theta que utilizaremos en funciones posteriores.

In [2]:
data = loadmat('ex4data1.mat')
y = data['y']
x = data['X']
y = np.array(y.T)[0]

weights = loadmat('ex4weights.mat')
theta1, theta2 = weights['Theta1'], weights['Theta2']
    
params = np.concatenate((np.ravel(theta1),np.ravel(theta2)))
num_entradas = theta1.shape[1]-1
num_ocultas = theta1.shape[0]
num_etiquetas = 10

Función de coste
---
Implementadas función sigmoide, de propagacion y de coste (sin regularizar).

In [3]:
def sigmoide(z):
    s = np.dot(z,-1)
    e = np.exp(s)
    d = 1 + e
    return 1/d

In [4]:
def coste(h,Y,num_etiquetas):  
    c = 0
    for i in range(num_etiquetas):
        y = (Y==i+1)*1
        c += sum(-y*(np.log(h[i])) - (1-y)*(np.log(1-h[i])))
        
    return c / Y.shape[0]

Cálculo del gradiente.
===
Cálculo de la derivada de la función sigmoide 

In [5]:
def deriv_sig(Z):
    sig = sigmoide(Z)
    return sig * (1 - sig)

Función que inicializa theta de forma aleatoria.

In [6]:
def pesosAleatorios(L_in, L_out):   
    e_ini = 0.12
    pesos = np.random.uniform(-e_ini,e_ini,size=(L_out, L_in+1))
    return pesos

Función auxiliar para la salida de la capa 2.

In [7]:
def prop(X,th1,th2): 
    a1 = sigmoide(np.dot(X, th1.T))
    a1 = np.concatenate((np.ones([a1.shape[0],1]),a1),axis=1)
    a2 = sigmoide(np.dot(a1, th2.T))
    
    return a2

Función de retro-propagación (regularizada)
---
Calcula todos los valores de salida (a) y los valores sigma (s) auxiliares para calcular el gradiente (grad) y su acumulación (delta), según las instrucciones dadas en la práctica. 

In [8]:
def retro_prop(X, Y, th1, th2, reg, num_etiquetas):    
    a1 = sigmoide(np.dot(X, theta1.T))
    a2 = np.concatenate((np.ones([a1.shape[0],1]),a1),axis=1)
    a3 = sigmoide(np.dot(a2, theta2.T))
    
    y = np.zeros((Y.shape[0],num_etiquetas))    
    for i in range (num_etiquetas):
        y[i, Y[i]-1] = 1 
        
    s3 = a3 - y
    s2 = np.dot(s3, th2)[:,1:]
    s2 = s2 * deriv_sig(np.dot(X, th1.T))
    delta1 = np.dot(s2.T, X)
    delta2 = np.dot(s3.T, a2)

    m = Y.shape[0]
    grad1 = delta1/m
    grad2 = delta2/m
    
    th1[:,1]=0
    th2[:,1]=0
    grad1 = grad1+(reg/m)*th1
    grad2 = grad2+(reg/m)*th2
    
    return np.concatenate((np.ravel(grad1),np.ravel(grad2))) 

Función de coste, añadiendo el término de regularización.

In [9]:
def coste_reg(h, Y, th1, th2, num_etiquetas, reg):
    m = Y.shape[0]
    th1[0] = 0
    th2[0] = 0
    c = coste(h,Y,num_etiquetas)
    c += (sum(sum(th1**2)) + sum(sum(th2**2))) * (reg/m*2)
       
    return c

Función de back-propagation, calcula el coste y el gradiente con las dos funciones anteriores, en versión regularizada. 

In [10]:
def backprop (params_rn, num_entradas, num_ocultas, num_etiquetas, X, Y, reg ):
    th1 = np.reshape(params_rn [:num_ocultas * (num_entradas + 1)], (num_ocultas, (num_entradas + 1)))
    th2 = np.reshape(params_rn [num_ocultas * (num_entradas + 1):], (num_etiquetas, (num_ocultas + 1)))
    
    X = np.concatenate((np.ones([X.shape[0],1]),X),axis=1)    
    
    p = prop(X,th1,th2).T

    coste = coste_reg(p, Y, th1, th2, num_etiquetas, reg)
    grad = retro_prop(X, Y, th1, th2, reg, num_etiquetas)

    return coste, grad

Aprendizaje de los parámetros de la red neuronal
---
Para entrenar a la red, utilizamos la función minimize y luego comprobamos el porcentaje de predicciones correctas con los datos dados.

In [11]:
def aprendNN(X, Y, params, num_entradas, num_ocultas, num_etiquetas, lamb):
    fmin = opt.minimize(fun=backprop, x0=params, 
                        args=(num_entradas, num_ocultas, num_etiquetas, X, Y, lamb),
                        method='TNC', jac=True, options={'maxiter':50})
    
    theta1 = np.reshape(fmin.x [:num_ocultas * (num_entradas + 1)], (num_ocultas, (num_entradas + 1)))
    theta2 = np.reshape(fmin.x [num_ocultas * (num_entradas + 1):], (num_etiquetas, (num_ocultas + 1)))
    
    X = np.concatenate((np.ones([X.shape[0],1]),X),axis=1)
    
    res = prop(X,theta1,theta2)   
    result = np.argmax(res,axis=1)     
    result = (result+1 == (Y))*1
    
    porcentaje = (sum(result)*100) / Y.shape[0]

    return porcentaje

In [12]:
aprendNN(x,y,params,num_entradas, num_ocultas, num_etiquetas, 1)

94.02